In [6]:
from __future__ import unicode_literals, print_function, division
from io import open
from nn_DataProcess import build_word2vec
from nn_Config import Config

if __name__ == '__main__':
    splist = []
    # 构建word2id词典
    word2id = {}
    with open(Config.word2id_path, encoding='utf-8') as f:
        for line in f.readlines():
            sp = line.strip().split()  # 去掉\n \t 等
            splist.append(sp)
        word2id = dict(splist)  # 转成字典

    # 转换索引的数据类型为整数
    for key in word2id:
        word2id[key] = int(word2id[key])


In [5]:
import torch
import torch.nn as nn
import netron
from model import LSTM_attention
from nn_Config import Config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 构建word2vec词向量
w2vec = build_word2vec(Config.pre_word2vec_path, word2id, None)
# 将词向量转化为Tensor
w2vec = torch.from_numpy(w2vec)
# CUDA接受float32，不接受float64
w2vec = w2vec.float()

# LSTM_attention
model = LSTM_attention(Config.vocab_size, Config.embedding_dim, w2vec, Config.update_w2v,
                       Config.hidden_dim, Config.num_layers, Config.drop_keep_prob, Config.n_class,
                       Config.bidirectional)

# 创建一个示例输入
input_example = torch.zeros((19998, 65), dtype=torch.long)

# 将模型转移到指定设备上
model = model.to(device)

# 将模型保存为ONNX格式
torch.onnx.export(model, input_example, "lstm_attention.onnx", export_params=True)

# 使用netron打开模型结构
netron.start("lstm_attention.onnx")

/Users/apple/opt/anaconda3/envs/Pytorch/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:3227: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


Serving 'lstm_attention.onnx' at http://localhost:8080


('localhost', 8080)